# dataclass 初探

参考：https://zhuanlan.zhihu.com/p/59885432

Python 3.7 引入了一个新的模块，这个模块就是今天要试探的 dataclass。

dataclass 的用法和普通的类装饰器没有任何区别，它的作用是替换定义类的时候的：`def __init__()`

我们来看看如何使用它

In [5]:
# 我们需要引入 dataclass 包
from dataclasses import dataclass 

@dataclass
class A:
    a: int
    b: int
    c: str
    d: str = "test"

a = A(1, 2, "3")
print(a)

A(a=1, b=2, c='3', d='test')


In [2]:
# 它的效果和以下代码完全一样
class A:
    def __init__(self, a, b, c, d="test"):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
a = A(1, 2, "3")
print(a)

使用了 dataclass 可以省下很多代码，可以帮我们节约很多时间，代码也变得很简洁了。



## 定义类型

我们发现，使用 dataclass 的时候，需要对初始化的参数进行类型定义，比如上面的例子里面，我为 a, b, c, d 定义的类型分别是 int, int, str 和 str。

那我建立实例的时候，传递非定义的类型的数据进去，会报错么？

答案是很明显的，是不会报错的，毕竟 python 是解释性语言嘛。如下例子：


In [6]:
a = A("name", "age", 123, 123)
print(a)

# cc：记着，这里需要重新运行一下 被dataclass修饰的A类

# 但是在 pycharm 之类的 IDE 里面，是会提醒修改的。

A(a='name', b='age', c=123, d=123)


那么我们可以使用万能的类型的么？当然是可以的，但是不建议（毕竟现在都建议写 python 的工程师加上类型检查了）

做法如下:

In [7]:
@dataclass
class A:
    a: ""
    b: 1
        

这样就可以随意传参了。

我们只需要随意给一个字符串就可以了，也可以是任何的其他类型

## 继承

使用了 dataclass 之后，类的继承还是之前的那样么？

我们来试试

In [9]:
@dataclass
class A:
    a: int
    b: str


@dataclass
class B(A):
    c: int
    d: int

b = B(a=1, b="2", c=3, d=4)
b

B(a=1, b='2', c=3, d=4)

就完了。

再来想想我们之前的继承 __init__ 是怎么写的

In [11]:
class A:
    def __init__(self, a: int, b: str):
        self.a = a
        self.b = b


class B(A):
    def __init__(self, a: int, b: str, c: int, d: int):
        super().__init__(a, b)
        self.c = c
        self.d = d

b = B(a=1, b="2", c=3, d=4)
b

一对比，是不是上面的代码简洁太多太多了！简直的优化利器！

## 使用 make_dataclass 快速创建类

除此之外，dataclasses 还提供了一个方法 make_dataclass 让我们可以快速创建类

In [14]:
from dataclasses import make_dataclass

A = make_dataclass(
    "A", 
    [("a", int), "b", ("c", str), ("d", int, 1)],
    namespace={'add_one': lambda self: self.a + 1})

a = A(1,2,3)
a

# 和下面代码意思一样的

A(a=1, b=2, c=3, d=1)

In [15]:
@dataclass
class A:
    a: int
    b: ""
    c: str
    d: int = 1

    def add_one(self):
        self.a += 1
        
a = A(1,2,3)
a
# cc： 那还是这个更清晰一些；

A(a=1, b=2, c=3, d=1)

## 小结

我们只是初步的使用 dataclass 来替换了以往的 __init__ 而已，还有很多新带入的东西没有使用，比如 field 还有其他的强大的功能。

见下文

# dataclass 浅析

参考链接：https://zhuanlan.zhihu.com/p/60009941


## field
field 在 dataclasses 里面是比较重要的功能， 用于初处理定义的参数非常有用
在 PEP 557 中是这样描述 field 的

`Field objects describe each defined field. These objects are created internally, and are returned by the fields() module-level method (see below). Users should never instantiate a Field object directly.`

大致意思就是 Field 对象是用于描述定义的字段的，这些对象是内部定义好了的。然后由 field() 方法返回，用户不用直接实例化 Field。
我们先看看 field 是如何使用的

In [3]:
from dataclasses import dataclass, field


@dataclass
class A:
    a: str = field(default="123")

可以用于设立默认值，和 a: str = "123" 一个效果，那为什么我们还需要 field 呢？
因为 field 的功能远不止这一个设置默认值，他还有很多有用的功能

### 1. 设置是否加载到 __init__ 里面去

In [18]:
@dataclass
class A:
    a: int
    b: int = field(default=10, init=False)
a = A(1) # 注意，实例化 A 的时候只需要一个参数，赋给 a 的
a

A(a=1, b=10)

In [20]:
# 等价于
class A:
    b = 10
    def __init__(self, a: int):
        self.a = a

a = A(1)
a
# cc：这里因为init=False，所以不会加入到init里面。

# 另外，可以看到通过filed定义和下面这段的定义，在实例化对象的时候输出是不一样的。

### 2.  设置是否成为 __repr__ 返回参数
我们在之前实例化 A 的时候，把实例化对象打印出来的话，是这样的：A(a=1, b=10)

那如果我们不想把特定的对象打印出来，可以这样写:

In [21]:
@dataclass
class A:
    a: int
    b: int = field(default=1, repr=False)

a = A(1)
print(a)
# 这时候，打印的结果为 A(a=1)

A(a=1)


### 3. 设置是否计算 hash 的对象之一
a: int = field(hash=False)
### 4. 设置是否成为和其他类进行对比的值之一
a: int = field(compare=False)

### 5. 定义 field 信息

In [7]:
from dataclasses import field, dataclass, fields
@dataclass
class A:
    a: int = field(metadata={"name": "a"}) # metadata 需要接受一个映射对象，也就是 python 的字典

metadata = fields(A)
print(metadata)


(Field(name='a',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x7fb260fba550>,default_factory=<dataclasses._MISSING_TYPE object at 0x7fb260fba550>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'name': 'a'}),_field_type=_FIELD),)


打印的结果如上。

以上输出是一个 tuple，第一个即是 a 字段的 field 定义。

可以通过 metadata[0].metadata["name"] 获取值

In [11]:
metadata[0].metadata["name"] 

'a'

### 6. 自定义处理定义的参数

有些字段需要我们进行一些预处理，不用传递初始值，由其他函数返回

我们可以这么写

In [13]:
def value():
    return "123"

@dataclass
class A:
    a: str = field(default_factory=value)

print(A().a) # 实例化 A 的时候已经可以不传递值了


123


In [14]:
A().a

'123'

## 使用 dataclass 设定初始方法

使用装饰器 dataclass 的时候，设定一些参数，即可选择是否需要这些初始方法

### 1. `__init__`

In [29]:
@dataclass(init=False)
class A:
    a: int = 1

print(A)


<class '__main__.A'>


### 2. __repr__

field 可以设置哪个参数不加入类返回值，设置
`@dataclass(repr=False)` 即可

### 3. __hash__
设置是否需要对类进行 hash，可以结合 `a: int = field(hash=True)` 一起设置

### 4. __eq__
这是类之间比较使用的方法，
同样可以结合 `a: int = field(compare=True)` 一起设置

## 源码剖析

dataclasses 这个库这么强大，我们来一步步剖析它的源码吧!

### field 源码剖析

In [ ]:
# 首先我们看看 field 的源码

def field(*, 
          default=MISSING, 
          default_factory=MISSING, 
          init=True, 
          repr=True,
          hash=None, 
          compare=True, 
          metadata=None):
    
    if default is not MISSING and default_factory is not MISSING:
        raise ValueError('cannot specify both default and default_factory')
        
    return Field(default, default_factory, init, repr, hash, compare, metadata)

这段代码很简单，对传入的参数进行判断之后，返回 Field 实例。

注意 default 和 default_factory 缺一不可，都是作为定义初始值的。

In [ ]:
# 然后我们来看看 Field 的源码：
class Field:
    __slots__ = ('name',
                 'type',
                 'default',
                 'default_factory',
                 'repr',
                 'hash',
                 'init',
                 'compare',
                 'metadata',
                 '_field_type',
                 )

    def __init__(self, default, default_factory, init, repr, hash, compare,
                 metadata):
        self.name = None
        self.type = None
        self.default = default
        self.default_factory = default_factory
        self.init = init
        self.repr = repr
        self.hash = hash
        self.compare = compare
        self.metadata = (_EMPTY_METADATA
                         if metadata is None or len(metadata) == 0 else
                         types.MappingProxyType(metadata))
        self._field_type = None

    def __repr__(self):
        return ('Field('
                f'name={self.name!r},'
                f'type={self.type!r},'
                f'default={self.default!r},'
                f'default_factory={self.default_factory!r},'
                f'init={self.init!r},'
                f'repr={self.repr!r},'
                f'hash={self.hash!r},'
                f'compare={self.compare!r},'
                f'metadata={self.metadata!r},'
                f'_field_type={self._field_type}'
                ')')

    def __set_name__(self, owner, name):
        func = getattr(type(self.default), '__set_name__', None)
        if func:
            # There is a __set_name__ method on the descriptor, call
            # it.
            func(self.default, owner, name)


简单的一个类，功能也就一个 __set_name__

我们注意一下 __repr__ 里面的有个细节:

f'name={self.name!r},', 比如 self.name 为 "name", 这里会返回 "name='name',"

### dataclass 源码剖析 

In [ ]:
def dataclass(_cls=None, *, init=True, repr=True, eq=True, order=False,
              unsafe_hash=False, frozen=False):

    def wrap(cls):
        return _process_class(cls, init, repr, eq, order, unsafe_hash, frozen)

    if _cls is None:
        return wrap
    return wrap(_cls)


这是一个很常见的装饰器

当我们定义类的时候，把类本身作为 _cls 参数传递进去，这时候返回一个 _process_class 函数的值

实例化类的时候，这时候 _cls 为 None, 返回 wrap 对象

接着我们来看 _process_class 源码

这段代码比较长，我们删减部分（不影响核心功能），删除的是生成初始化函数的部分，有兴趣的读者可以自己查看一下。


In [ ]:
def _process_class(cls, init, repr, eq, order, unsafe_hash, frozen):
    fields = {}

    setattr(cls, _PARAMS, _DataclassParams(init, repr, eq, order,
                                           unsafe_hash, frozen))
    any_frozen_base = False
    has_dataclass_bases = False
    for b in cls.__mro__[-1:0:-1]:
        base_fields = getattr(b, _FIELDS, None)
        if base_fields:
            has_dataclass_bases = True
            for f in base_fields.values():
                fields[f.name] = f
            if getattr(b, _PARAMS).frozen:
                any_frozen_base = True
    cls_annotations = cls.__dict__.get('__annotations__', {})
    cls_fields = [_get_field(cls, name, type)
                  for name, type in cls_annotations.items()]
    for f in cls_fields:
        fields[f.name] = f
        if isinstance(getattr(cls, f.name, None), Field):
            if f.default is MISSING:
                delattr(cls, f.name)
            else:
                setattr(cls, f.name, f.default)
    setattr(cls, _FIELDS, fields)

    if init:
        has_post_init = hasattr(cls, _POST_INIT_NAME)
        flds = [f for f in fields.values()
                if f._field_type in (_FIELD, _FIELD_INITVAR)]
        _set_new_attribute(cls, '__init__',
                           _init_fn(flds,
                                    frozen,
                                    has_post_init,
                                    '__dataclass_self__' if 'self' in fields
                                            else 'self',
                          ))

    return cls

## 尾声
从功能上来看，dataclass 为我们带来了比较好优化类方案，提供的各类方法也足够用，可以在之后的项目里面逐渐使用起来。
从源码上来看，源码整体比较简洁，使用了比较少见的 __annotations__，技巧足够，代码简单易学。
建议新手可以从此入手，即可学习装饰器也可学习优秀代码。

